In [1]:
# include <iostream>
# include <vector>
# include <stdexcept>

# Une classe pour les matrices

De nombreux problèmes d'EDPs conduisent à la résolution d'une équation de type $Ax=b$ où $A$ est une matrice carrée inversible. Nous devons donc créer une classe `matrix` permettant le stockage de ces matrices, ainsi que des fonctions pour la résolution des systèmes.

In [2]:
%%file matrix.hpp

/*****************************************************************************************
*
* class matrix
*
*****************************************************************************************/
class matrix
{
private:
    std::size_t nx, ny;
    std::vector<double> array;
public:
    matrix() = default;                         // constructeur sans paramètre
    matrix(const matrix&) = default;            // constructeur par copie
    matrix& operator=(const matrix&) = default; // assignation par copie
    matrix(matrix&&) = default;                 // constructeur par déplacement
    matrix& operator=(matrix&&) = default;      // assignation par déplacement
    matrix(const std::size_t nx, const std::size_t ny): nx{nx}, ny{ny}
    {
        array.resize(nx*ny);
    }

    // shape of the matrix
    auto shape() const
    {
        return std::make_pair(nx, ny);
    }
    
    // fill the matrix with a constant value
    void fill(const double value)
    {
        std::fill(array.begin(), array.end(), value);
    }
    
    // modify the value: the matrix becomes identity
    void identity()
    {
        std::fill(array.begin(), array.end(), 0);
        for (std::size_t i = 0; i < nx; ++i)
            array[i + ny*i] = 1;
    }
    
    // return the value ith row and jth column: read data
    double operator()(const std::size_t i, const std::size_t j) const
    {
        return array[j + ny*i];
    }
    
    // return the reference ith row and jth column: write data
    double& operator()(const std::size_t i, const std::size_t j)
    {
        return array[j + ny*i];
    }
    
    // operators overload 
    matrix& operator+=(const matrix);
    matrix& operator-=(const matrix);
    matrix& operator*=(const matrix);
    matrix& operator^=(const std::size_t);
    matrix& operator+=(const double);
    matrix& operator-=(const double);
    matrix& operator*=(const double);
    matrix& operator/=(const double);
    
    bool is_zero(double pres = 1.e-14)
    {
        for (auto a: array)
            if (std::fabs(a) > pres)
                return false;
        return true;
    }
};

std::ostream& operator<<(std::ostream &out, const matrix &mat)
{
    out.setf(std::ios_base::scientific);
    out.precision(3);
    auto shape = mat.shape();
    for(std::size_t i=0; i<shape.first; ++i)
    {
        out << "[ ";
        for(std::size_t j=0; j<shape.second; ++j)
            std::cout << std::setw(10) << mat(i, j) << " ";
        out << "]\n";
    }
    out.unsetf(std::ios_base::scientific);
    return out;
}

matrix& matrix::operator+=(const matrix B)
{
    if (shape()!=B.shape())
        throw std::length_error("Dimension mismatch !!");
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] += B.array[i];
    return *this;
}

matrix& matrix::operator-=(const matrix B)
{
    if (shape()!=B.shape())
        throw std::length_error("Dimension mismatch !!");
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] -= B.array[i];
    return *this;
}

matrix& matrix::operator*=(const matrix B)
{
    auto shapeA = shape();
    auto shapeB = B.shape();
    if (shapeA.second != shapeB.first)
        throw std::length_error("Dimension mismatch !!");
    matrix Acopy=*this;
    ny = shapeB.second;
    array.resize(nx*ny);
    for(std::size_t i=0; i<nx; ++i)
        for(std::size_t j=0; j<ny; ++j)
        {
            double som = 0;
            for(std::size_t k=0; k<shapeA.second; ++k)
                som += Acopy(i, k)*B(k, j);
            (*this)(i, j) = som;
        }
    return *this;
}

matrix& matrix::operator^=(const std::size_t p)
{
    if (nx!=ny)
        throw std::length_error("Matrix has to be square !!");
    matrix s = *this; // copy of the matrix
    // modify the matrix into identity
    identity();
    std::size_t m = p;
    while (m>0)
    {
        if (m%2 != 0)
        *this *= s;
        s *= s;
        m /= 2;
    }
    return *this;
}

matrix& matrix::operator+=(const double a)
{
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] += a;
    return *this;
}

matrix& matrix::operator-=(const double a)
{
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] -= a;
    return *this;
}

matrix& matrix::operator*=(const double a)
{
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] *= a;
    return *this;
}

matrix& matrix::operator/=(const double a)
{
    if (a == 0.)
        throw std::invalid_argument("Division by 0.!!");
    for(std::size_t i=0; i<array.size(); ++i)
        array[i] /= a;
    return *this;
}

matrix operator+(const matrix& A, const matrix& B)
{
    matrix C = A;
    return C += B;
}

matrix operator-(const matrix& A, const matrix& B)
{
    matrix C = A;
    return C -= B;
}

matrix operator^(const matrix& A, const std::size_t p)
{
    matrix C = A;
    return C^=p;
}

matrix operator*(const matrix& A, const matrix& B)
{
    matrix C = A;
    return C *= B;
}

matrix identity(const std::size_t nx, const std::size_t ny, const int k = 0)
{
    // BEGIN SOLUTION
    auto A = matrix(nx, ny);
    A.fill(0);
    std::size_t i = std::max(0, -k), j = std::max(0, k);
    while ((i < nx) && (j < ny))
        A(i++,j++) = 1;
    return A;
    // END SOLUTION
}

std::vector<double> operator*(const matrix& A, const std::vector<double>& v)
{
    auto shapeA = A.shape();
    if (shapeA.second != v.size())
        throw std::length_error("Dimension mismatch !!");
    std::vector<double> vout(shapeA.first);
    for(std::size_t i=0; i<shapeA.first; ++i)
    {
        double som = 0;
        for(std::size_t j=0; j<v.size(); ++j)
            som += A(i, j) * v[j];
        vout[i] = som;
    }
    return vout;
}

std::vector<double> operator*(const std::vector<double>& v, const matrix& A)
{
    auto shapeA = A.shape();
    if (shapeA.first != v.size())
        throw std::length_error("Dimension mismatch !!");
    std::vector<double> vout(shapeA.second);
    for(std::size_t j=0; j<shapeA.second; ++j)
    {
        double som = 0;
        for(std::size_t i=0; i<v.size(); ++i)
            som += v[i] * A(i, j);
        vout[j] = som;
    }
    return vout;
}

double operator*(const std::vector<double>& v, const std::vector<double>& w)
{
    if (v.size() != w.size())
        throw std::length_error("Dimension mismatch !!");
    return std::inner_product(v.begin(), v.end(), w.begin(), 0.);
}

matrix operator+(const double a, const matrix& A)
{
    matrix B = A;
    return B += a;
}

matrix operator+(const matrix& A, const double a)
{
    matrix B = A;
    return B += a;
}

matrix operator-(const double a, const matrix& A)
{
    matrix B = A;
    return B -= a;
}

matrix operator-(const matrix& A, const double a)
{
    matrix B = A;
    return B -= a;
}

matrix operator*(const double a, const matrix& A)
{
    matrix B = A;
    return B *= a;
}

matrix operator*(const matrix& A, const double a)
{
    matrix B = A;
    return B *= a;
}

matrix operator/(const matrix& A, const double a)
{
    matrix B = A;
    return B /= a;
}

Overwriting matrix.hpp


In [3]:
#include "matrix.hpp"

#### Question 1

Completez la fonction `identity` afin de pouvoir exécuter la cellule suivante. C'est-à-dire que la fonction doit avoir le même comportement que la fonction `eye` du module `numpy` de `python` : prendre deux entiers $N$ et $M$ en paramètres ainsi qu'un paramètre optionnel $k$ (dont la valeur par défaut devra être $0$) et retourner la matrice de taille $N{\times}M$ remplie de $0$ sauf sur la diagonale indexées par $k$.

In [4]:
{
    auto N = 5;
    auto A = 2.*identity(N,N) - identity(N,N,1) - identity(N,N,-1);
    std::cout << A;
}

[  2.000e+00 -1.000e+00  0.000e+00  0.000e+00  0.000e+00 ]
[ -1.000e+00  2.000e+00 -1.000e+00  0.000e+00  0.000e+00 ]
[  0.000e+00 -1.000e+00  2.000e+00 -1.000e+00  0.000e+00 ]
[  0.000e+00  0.000e+00 -1.000e+00  2.000e+00 -1.000e+00 ]
[  0.000e+00  0.000e+00  0.000e+00 -1.000e+00  2.000e+00 ]


# Résolution des systèmes linéaires

Nous ajoutons des fonctions permettant de résoudre un système linéaire de la forme $Ax=b$ où la matrice $A$ est une matrice carrée inversible.

In [5]:
%%file lu.hpp

/*****************************************************************************************
*
* lu decomposition
*
*****************************************************************************************/

matrix lu(const matrix& A)
{
    auto n = A.shape().first, m = A.shape().second;
    if (n != m)
        throw std::length_error("Error in lu decomposition: dimension mismatch !!");
    matrix LU = matrix(n,n);
    for (auto j=0; j<n; ++j)
    {
        for (auto i=0; i<=j; ++i)
        {
            auto sum = 0.;
            for (auto k=0; k<i; ++k)
                sum += LU(i,k) * LU(k,j);
            LU(i,j) = A(i,j) - sum;
        }
        if (LU(j,j) == 0)
            throw std::invalid_argument("Error in lu decomposition: the decomposition failed !!");
        for (auto i=j+1; i<n; ++i)
        {
            auto sum = 0.;
            for (auto k=0; k<j; ++k)
                sum += LU(i,k) * LU(k,j);
            LU(i,j) = (A(i,j) - sum) / LU(j,j);
        }
    }
    return LU;
}

bool lu_verification(const matrix& A, const matrix& LU)
{
    auto shapeA = A.shape(), shapeLU = LU.shape();
    auto n = shapeA.first;
    if ((n != shapeA.second) || (n != shapeLU.first) || (n != shapeLU.second))
        throw std::length_error("Error in lu_verification: dimension mismatch !!");
    auto L = identity(n,n);
    for (auto i=0; i<n; ++i)
        for (auto j=0; j<i; ++j)
            L(i,j) = LU(i,j);
    auto U = matrix(n,n);
    for (auto i=0; i<n; ++i)
        for (auto j=i; j<n; ++j)
            U(i,j) = LU(i,j);
    auto test = A - L*U;
    return test.is_zero();
}

std::vector<double> solve_lu(const matrix& LU, const std::vector<double>& b)
{
    auto n = LU.shape().first;
    if ((n != LU.shape().second) || (n != b.size()))
        throw std::length_error("Error in solve_lu: dimension mismatch !!");
    std::vector<double> x(n);
    // solve Ly=b
    for (auto i=0; i<n; ++i)
    {
        auto sum = b[i];
        for (auto j=0; j<i; ++j)
            sum -= LU(i,j) * x[j];
        x[i] = sum;
    }
    // solve Ux=y
    for (int i=n-1; i>=0; --i)
    {
        auto sum = x[i];
        for (auto j=i+1; j<n; ++j)
            sum -= LU(i,j) * x[j];
        x[i] = sum / LU(i,i);
    }
    return x;
}

Overwriting lu.hpp


In [6]:
#include "lu.hpp"

#### Question 2

Vérifiez que la décomposition $LU$ fonctionne correctement en exécutant la cellule suivante. Vous devriez trouver

```
[  2.000e+00 -1.000e+00  0.000e+00  0.000e+00  0.000e+00 ]
[ -5.000e-01  1.500e+00 -1.000e+00  0.000e+00  0.000e+00 ]
[  0.000e+00 -6.667e-01  1.333e+00 -1.000e+00  0.000e+00 ]
[  0.000e+00  0.000e+00 -7.500e-01  1.250e+00 -1.000e+00 ]
[  0.000e+00  0.000e+00  0.000e+00 -8.000e-01  1.200e+00 ]
```

Vérifiez ensuite avec la cellule suivante : après avoir créé de manière aléatoire une matrice $L$, une matrice $U$, on effectue le produit $L{\times}U$ puis l'on calcule la décomposition $LU$ du résultat.

_Remarque : la décomposition $LU$ proposée ne retourne qu'une seule matrice pour économiser de la place. Les deux matrices $L$ et $U$ sont mises dans la même matrice..._

In [7]:
{
    auto N = 5;
    auto A = 2.*identity(N,N) - identity(N,N,1) - identity(N,N,-1);
    auto LU = lu(A);
    std::cout << LU;
}

[  2.000e+00 -1.000e+00  0.000e+00  0.000e+00  0.000e+00 ]
[ -5.000e-01  1.500e+00 -1.000e+00  0.000e+00  0.000e+00 ]
[  0.000e+00 -6.667e-01  1.333e+00 -1.000e+00  0.000e+00 ]
[  0.000e+00  0.000e+00 -7.500e-01  1.250e+00 -1.000e+00 ]
[  0.000e+00  0.000e+00  0.000e+00 -8.000e-01  1.200e+00 ]


In [8]:
#include <random>

In [9]:
{
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(-1, 1);
    auto n = 5;
    auto L = identity(n,n), U = matrix(n,n);
    for (auto i=0; i<n; ++i)
    {
        L(i,i) = 1.;
        U(i,i) = dis(gen);
        for (auto j=0; j<i; ++j)
        {
            L(i,j) = dis(gen);
            U(j,i) = dis(gen);
        }
    }
    std::cout << "L = \n" << L << std::endl;
    std::cout << "U = \n" << U << std::endl;
    auto A = L * U;
    std::cout << "LU = \n" << lu(A) << std::endl;
}

L = 
[  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 ]
[ -5.569e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00 ]
[ -5.288e-01 -5.057e-01  1.000e+00  0.000e+00  0.000e+00 ]
[ -5.457e-01  8.040e-01  9.833e-01  1.000e+00  0.000e+00 ]
[ -1.868e-01  6.149e-01 -3.085e-01  7.228e-01  1.000e+00 ]

U = 
[  1.716e-01  7.951e-01  2.792e-01 -7.485e-01  4.542e-01 ]
[  0.000e+00 -8.086e-01  5.876e-01 -1.747e-01  2.708e-01 ]
[  0.000e+00  0.000e+00 -6.455e-01  6.114e-01  1.753e-02 ]
[  0.000e+00  0.000e+00  0.000e+00  6.152e-01 -6.248e-01 ]
[  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -2.997e-01 ]

LU = 
[  1.716e-01  7.951e-01  2.792e-01 -7.485e-01  4.542e-01 ]
[ -5.569e-01 -8.086e-01  5.876e-01 -1.747e-01  2.708e-01 ]
[ -5.288e-01 -5.057e-01 -6.455e-01  6.114e-01  1.753e-02 ]
[ -5.457e-01  8.040e-01  9.833e-01  6.152e-01 -6.248e-01 ]
[ -1.868e-01  6.149e-01 -3.085e-01  7.228e-01 -2.997e-01 ]



#### Question 3

Vérifiez à présent que la fonction de résolution des sytèmes linéaires par la méthode de la décomposition $LU$ fonctionne. Dans la cellule suivante, une matrice $A$ et un vecteur $b$ aléatoires sont créés. Complétez la cellule afin de résoudre le système linéaire $Ax=b$, d'afficher le résultat ainsi que $b$ et $Ax$ afin de vérifier que le système est bien résolu.

In [11]:
{
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(0, 1);
    auto n = 8;
    auto A = matrix(n,n);
    std::vector<double> b(n);
    for (auto i=0; i<n; ++i)
    {
        b[i] = dis(gen);
        for (auto j=0; j<n; ++j)
            A(i,j) = dis(gen);
    }
    // BEGIN SOLUTION
    auto x = solve_lu(lu(A), b);
    auto y = A*x;
    std::cout << "x  = [ ";
    std::copy(x.begin(), x.end(), std::ostream_iterator<double>(std::cout, " "));
    std::cout << "]"<< std::endl;
    std::cout << "Ax = [ ";
    std::copy(y.begin(), y.end(), std::ostream_iterator<double>(std::cout, " "));
    std::cout << "]"<< std::endl;
    std::cout << "b  = [ ";
    std::copy(b.begin(), b.end(), std::ostream_iterator<double>(std::cout, " "));
    std::cout << "]"<< std::endl;
    // END SOLUTION
}

x  = [ 1.44 -0.0966 1.74 -0.483 0.0429 -0.561 -0.827 0.209 ]
Ax = [ 0.95 0.613 0.0267 0.782 0.757 0.99 0.000254 0.845 ]
b  = [ 0.95 0.613 0.0267 0.782 0.757 0.99 0.000254 0.845 ]
